# OpenShift Lightspeed Integration

## Overview
This notebook demonstrates integration with OpenShift Lightspeed (OLS), the AI-powered assistant for OpenShift. It configures OLS, sends queries, and uses AI-powered insights for intelligent operations.

## 📋 Prerequisites

**IMPORTANT**: This notebook requires pre-configured infrastructure. Do NOT proceed without completing setup.

### Required Infrastructure

1. **MCP Server Deployed** - Go-based standalone MCP server
   - **Deployment guide**: See [How-To: Deploy MCP Server & Lightspeed](../../docs/how-to/deploy-mcp-server-lightspeed.md)
   - **Architecture**: [ADR-036: Go-Based Standalone MCP Server](../../docs/adrs/036-go-based-standalone-mcp-server.md)
   - **Expected service**: `mcp-server.self-healing-platform.svc:8080`
   - **Verify**: `oc get deployment mcp-server -n self-healing-platform`

2. **OpenShift Lightspeed Deployed** - AI assistant with OLSConfig
   - **Configuration**: See [How-To: Deploy MCP Server & Lightspeed](../../docs/how-to/deploy-mcp-server-lightspeed.md)
   - **Integration**: [ADR-016: OpenShift Lightspeed OLSConfig Integration](../../docs/adrs/016-openshift-lightspeed-olsconfig-integration.md)
   - **Required**: LLM provider secret (OpenAI, Gemini, or Claude)
   - **Verify**: `oc get olsconfig cluster`

3. **Coordination Engine Running** - Go-based remediation engine
   - **Architecture**: [ADR-038: Go Coordination Engine Migration](../../docs/adrs/038-go-coordination-engine-migration.md)
   - **Expected service**: `coordination-engine.self-healing-platform.svc:8080`
   - **Verify**: `oc get deployment coordination-engine -n self-healing-platform`

4. **Previous Notebook Completed**
   - Complete `mcp-server-integration.ipynb` first
   - MCP client working and validated

### Quick Setup Check

```bash
# Verify all services are running
oc get deployment mcp-server coordination-engine -n self-healing-platform
oc get olsconfig cluster
```

**Complete deployment guide**: [How-To: Deploy MCP Server & Lightspeed](../../docs/how-to/deploy-mcp-server-lightspeed.md)

---

## 🚀 OpenShift Lightspeed Setup

**Lightspeed requires manual configuration.** Follow the deployment guide for complete setup instructions.

📖 **IMPORTANT**: See [How-To: Deploy MCP Server and Configure OpenShift Lightspeed](../../docs/how-to/deploy-mcp-server-lightspeed.md)

### Setup Steps Overview

1. **Install OpenShift Lightspeed Operator** (from OperatorHub)
2. **Create LLM Provider API Key Secret** (OpenAI, Gemini, IBM BAM, or Azure)
3. **Create OLSConfig Resource** (customize YAML from deployment guide)
4. **Verify Integration** (check OLSConfig status and console plugin)

**The deployment guide provides:**
- Complete OLSConfig YAML examples for all providers
- Secret creation commands for each LLM provider
- MCP server integration configuration
- Troubleshooting steps and validation

⚠️ **Do NOT proceed with this notebook until Lightspeed is configured!**

Check deployment status:
```bash
# Verify OpenShift Lightspeed is installed
oc get olsconfig cluster
oc describe olsconfig cluster

# Check console plugin is enabled
oc get console.operator cluster -o jsonpath='{.spec.plugins}' | grep lightspeed
```

---

## Learning Objectives
- Configure OpenShift Lightspeed
- Send queries to Lightspeed
- Process AI-powered responses
- Integrate insights with self-healing platform
- Implement AI-driven operations

## Key Concepts
- **OpenShift Lightspeed**: AI assistant for OpenShift
- **OLSConfig**: Configuration for Lightspeed with MCP server integration
- **Query Processing**: Send questions to Lightspeed
- **Response Analysis**: Extract insights from responses
- **AI-Powered Operations**: Use AI for decision making

## Setup Section

In [ ]:
import sys
import os
import json
import logging
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import requests
from typing import Dict, List, Any

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

# Try to import common functions, with fallback
try:
    from common_functions import setup_environment
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup environment
env_info = setup_environment()
logger.info(f"Environment ready: {env_info}")

# Define paths
DATA_DIR = Path('/opt/app-root/src/data')
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Configuration
OLS_SERVER_URL = os.getenv('OLS_SERVER_URL', 'http://ols-server:8000')
NAMESPACE = 'self-healing-platform'
REQUEST_TIMEOUT = 30

logger.info(f"OpenShift Lightspeed integration initialized")
logger.info(f"OLS Server URL: {OLS_SERVER_URL}")

## Implementation Section

### 1. Configure OpenShift Lightspeed

In [ ]:
# OLSConfig resource
ols_config = {
    'apiVersion': 'ols.openshift.io/v1alpha1',
    'kind': 'OLSConfig',
    'metadata': {
        'name': 'cluster',
        'namespace': 'openshift-lightspeed'
    },
    'spec': {
        'llm': {
            'providers': [
                {
                    'name': 'openai',
                    'type': 'openai',
                    'url': 'https://api.openai.com/v1',
                    'credentials': {
                        'secretRef': {
                            'name': 'openai-credentials',
                            'namespace': 'openshift-lightspeed'
                        }
                    }
                }
            ],
            'model': 'gpt-4',
            'temperature': 0.7,
            'max_tokens': 2000
        },
        'rag': {
            'enabled': True,
            'vectorstore': {
                'type': 'elasticsearch',
                'url': 'https://elasticsearch:9200'
            }
        },
        'logging': {
            'level': 'INFO',
            'format': 'json'
        }
    }
}

logger.info(f"OLSConfig configured: {ols_config['spec']['llm']['model']}")
print(json.dumps(ols_config, indent=2))

### 2. Query OpenShift Lightspeed

In [ ]:
class LightspeedClient:
    """Client for OpenShift Lightspeed communication."""
    
    def __init__(self, server_url, timeout=30):
        self.server_url = server_url
        self.timeout = timeout
        self.session = requests.Session()
    
    def query(self, question: str, context: Dict[str, Any] = None) -> Dict[str, Any]:
        """Send query to Lightspeed."""
        try:
            payload = {
                'question': question,
                'context': context or {}
            }
            
            response = self.session.post(
                f"{self.server_url}/query",
                json=payload,
                timeout=self.timeout
            )
            
            if response.status_code == 200:
                logger.info(f"Query processed successfully")
                return response.json()
            else:
                logger.error(f"Query failed: {response.status_code}")
                return {'error': response.text}
        except Exception as e:
            logger.error(f"Query error: {e}")
            return {'error': str(e)}
    
    def get_recommendations(self, issue_type: str, context: Dict[str, Any]) -> Dict[str, Any]:
        """Get AI recommendations for issue."""
        try:
            payload = {
                'issue_type': issue_type,
                'context': context
            }
            
            response = self.session.post(
                f"{self.server_url}/recommendations",
                json=payload,
                timeout=self.timeout
            )
            
            if response.status_code == 200:
                logger.info(f"Recommendations retrieved")
                return response.json()
            else:
                logger.error(f"Recommendations failed: {response.status_code}")
                return {'error': response.text}
        except Exception as e:
            logger.error(f"Recommendations error: {e}")
            return {'error': str(e)}

# Initialize Lightspeed client
lightspeed_client = LightspeedClient(OLS_SERVER_URL)

# Send queries
query1 = lightspeed_client.query(
    "How do I troubleshoot high CPU usage in OpenShift pods?",
    {'namespace': NAMESPACE}
)
logger.info(f"Query 1 response: {json.dumps(query1, indent=2, default=str)[:200]}...")

query2 = lightspeed_client.query(
    "What are best practices for pod resource limits?",
    {'namespace': NAMESPACE}
)
logger.info(f"Query 2 response: {json.dumps(query2, indent=2, default=str)[:200]}...")

### 3. Get AI Recommendations

In [ ]:
# Get recommendations for different issue types
issue_context = {
    'pod_name': 'coordination-engine-0',
    'namespace': NAMESPACE,
    'cpu_usage': 85,
    'memory_usage': 72,
    'restart_count': 2
}

recommendations = lightspeed_client.get_recommendations(
    'high_resource_usage',
    issue_context
)

logger.info(f"Recommendations retrieved")
print("\nAI Recommendations:")
print(json.dumps(recommendations, indent=2, default=str))

### 4. Process Lightspeed Responses

In [ ]:
def extract_insights(response: Dict[str, Any]) -> Dict[str, Any]:
    """
    Extract actionable insights from Lightspeed response.
    
    Args:
        response: Lightspeed response
    
    Returns:
        Extracted insights
    """
    try:
        if 'error' in response:
            logger.error(f"Response error: {response['error']}")
            return {'status': 'error', 'message': response['error']}
        
        # Extract key information
        insights = {
            'status': 'success',
            'timestamp': datetime.now().isoformat(),
            'answer': response.get('answer', ''),
            'confidence': response.get('confidence', 0.0),
            'sources': response.get('sources', []),
            'recommendations': response.get('recommendations', [])
        }
        
        logger.info(f"Insights extracted (confidence: {insights['confidence']:.2%})")
        return insights
    except Exception as e:
        logger.error(f"Insight extraction error: {e}")
        return {'status': 'error', 'message': str(e)}

# Extract insights from responses
insights1 = extract_insights(query1)
insights2 = extract_insights(query2)
insights3 = extract_insights(recommendations)

print("\nExtracted Insights:")
print(f"Query 1 - Confidence: {insights1.get('confidence', 0):.2%}")
print(f"Query 2 - Confidence: {insights2.get('confidence', 0):.2%}")
print(f"Recommendations - Confidence: {insights3.get('confidence', 0):.2%}")

### 5. Track Lightspeed Integration

In [ ]:
# Create Lightspeed integration tracking dataframe
lightspeed_tracking = pd.DataFrame([
    {
        'timestamp': datetime.now().isoformat(),
        'query_type': np.random.choice(['troubleshooting', 'best_practices', 'recommendations']),
        'issue_type': np.random.choice(['high_cpu', 'high_memory', 'pod_crash', 'network']),
        'confidence': np.random.uniform(0.7, 0.99),
        'response_time_ms': np.random.randint(500, 3000),
        'actionable': np.random.choice([True, True, True, False])
    }
    for _ in range(20)  # Simulate 20 Lightspeed queries
])

# Save tracking data
tracking_file = PROCESSED_DIR / 'lightspeed_integration_tracking.parquet'
lightspeed_tracking.to_parquet(tracking_file)

logger.info(f"Saved Lightspeed integration tracking data")
print(lightspeed_tracking.to_string())

## Validation Section

In [ ]:
# Verify outputs
assert tracking_file.exists(), "Lightspeed tracking file not created"

avg_confidence = lightspeed_tracking['confidence'].mean()
actionable_rate = lightspeed_tracking['actionable'].sum() / len(lightspeed_tracking)
avg_response_time = lightspeed_tracking['response_time_ms'].mean()

logger.info(f"✅ All validations passed")
print(f"\nOpenShift Lightspeed Integration Summary:")
print(f"  Queries Processed: {len(lightspeed_tracking)}")
print(f"  Average Confidence: {avg_confidence:.2%}")
print(f"  Actionable Responses: {actionable_rate:.1%}")
print(f"  Average Response Time: {avg_response_time:.0f}ms")
print(f"\nQuery Type Distribution:")
print(lightspeed_tracking['query_type'].value_counts())

## Integration Section

This notebook integrates with:
- **Input**: Cluster issues and troubleshooting questions
- **Output**: AI-powered insights and recommendations
- **Monitoring**: Query confidence and response times
- **Next**: LlamaStack integration

## Next Steps

1. Monitor Lightspeed query performance
2. Proceed to `llamastack-integration.ipynb`
3. Deploy LlamaStack for local LLM inference
4. Combine MCP, Lightspeed, and LlamaStack
5. Complete Phase 6 implementation

## References

### Deployment Guides
- [Deploy MCP Server + Lightspeed](../../docs/how-to/deploy-mcp-server-lightspeed.md)
- [MCP-Lightspeed Configuration](../../docs/MCP-LIGHTSPEED-CONFIGURATION.md)

### Architecture Decision Records
- [ADR-016: OpenShift Lightspeed OLSConfig Integration](../../docs/adrs/016-openshift-lightspeed-olsconfig-integration.md)
- [ADR-036: Go-Based Standalone MCP Server](../../docs/adrs/036-go-based-standalone-mcp-server.md)
- [ADR-038: Go Coordination Engine Migration](../../docs/adrs/038-go-coordination-engine-migration.md)
- [ADR-002: Hybrid Deterministic-AI Self-Healing](../../docs/adrs/002-hybrid-self-healing-approach.md)
- [ADR-012: Notebook Architecture](../../docs/adrs/012-notebook-architecture-end-to-end-workflows.md)

### External Documentation
- [OpenShift Lightspeed](https://docs.openshift.com/container-platform/latest/lightspeed/)
- [OLSConfig Reference](https://docs.openshift.com/container-platform/latest/lightspeed/olsconfig/)
- [Model Context Protocol](https://modelcontextprotocol.io/)